# BIDDING STATERGIES

## Approach 1

In this approach, we just bid the predicted demand after subtracting the predicted solar output at the predicted market price.

In [1]:
import numpy as np
import pandas as pd
from scorer import bill_maker

In [2]:
demand=pd.read_csv("Prediction/demand.csv", header=None)
market_price=pd.read_csv("Prediction/market_price.csv", header=None)
solar_output=pd.read_csv("Prediction/solar_output.csv", header=None)

In [3]:
bid_quantity_pred=pd.DataFrame(demand-solar_output)

In [4]:
bid_price_pred=pd.DataFrame(market_price)

### Comparing prediction with Oracle

In [5]:
demand_oracle=pd.read_csv("Processed Data/Demand_Val_pred.csv", header=None)
market_price_oracle=pd.read_csv("Processed Data/Price_Val_pred.csv", header=None)
solar_output_oracle=pd.read_csv("Processed Data/Solar_Val_pred.csv", header=None)

In [6]:
bid_quantity_oracle=pd.DataFrame(demand_oracle - solar_output_oracle)

In [7]:
bid_price_oracle=pd.DataFrame(market_price_oracle)

In [8]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

In [9]:
daily_bill_oracle,total_bill_oracle=bill_maker(bid_price_oracle,bid_quantity_oracle)

In [10]:
total_bill_pred

383320.09645041835

In [11]:
total_bill_oracle

418580.69730000006

### Saving prediction to output

In [12]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)

## Approach 2

In this approach, we try to find the optimal margin that should be added to market price so that we avoid loosing the bid. We try to find this margin by considering the full training data.

In [13]:
demand_actual_train=pd.read_csv("Processed Data/Demand_Train.csv", header=None)
market_price_train=pd.read_csv("Processed Data/Price_Train.csv", header=None)
market_price_train_oracle=pd.read_csv("Processed Data/Price_Train_pred.csv", header=None)
solar_output_train=pd.read_csv("Processed Data/Solar_Train.csv", header=None)

In [14]:
pd.DataFrame(market_price_train.describe()[1:2].mean()).describe()

,0
count,24.000000
mean,2.904113
std,0.950572
min,1.577847
25%,2.067374
50%,3.015271
75%,3.423165
max,4.616106


In [15]:
pd.DataFrame(market_price.describe()[1:2].mean()).describe()

,0
count,24.000000
mean,2.989362
std,1.005487
min,1.603741
25%,2.102923
50%,3.115149
75%,3.523238
max,4.831434


In [16]:
pd.DataFrame(market_price.describe()[1:2].mean()).describe().loc[["mean"]]-pd.DataFrame(market_price_train.describe()[1:2].mean()).describe().loc[["mean"]]

,0
mean,0.085249


From the above difference of mean, we can see that mean of predicted market price is greater than that of the training data, Hence we subtract this mean from the predicted market price.

In [17]:
bid_price_pred=pd.DataFrame(market_price - 0.085249)

In [18]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [19]:
total_bill_pred

473192.10357479547

In [20]:
total_bill_oracle

418580.69730000006

We can also find hour wise mean for the training data and correspondingly add/subtract from our predicted market price.

In [21]:
hourly_mean=(market_price.describe().loc[["mean"]] - market_price_train.describe().loc[["mean"]]).mean()

In [22]:
bid_price_pred=pd.DataFrame(market_price - hourly_mean)

In [23]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [24]:
total_bill_pred

454251.62216230325

In [25]:
total_bill_oracle

418580.69730000006

We can also subtract the 25th percentile of the difference of price predicted by oracle and actual price when the predicted price by Oracle is less than Actual price (the case when we loose the bid) to ensure that 75% the we win it.

In [26]:
margin_25=(market_price_train_oracle - market_price_train)[(market_price_train_oracle - market_price_train)<0].describe().loc[['25%']].mean()

In [27]:
bid_price_pred=pd.DataFrame(market_price - margin_25)

In [28]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [29]:
total_bill_pred

309519.2878948272

In [30]:
total_bill_oracle

418580.69730000006

### Saving prediction to output

In [31]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)